In [ ]:
import pandas as pd
import numpy as np

from src.all_in_one import *

In [ ]:
processed_data_path = path_manager.processed_data

In [ ]:
# Creating a list of all rid across all datasets
rid = pd.DataFrame()  # An empty dataframe
if os.path.exists(processed_data_path) and os.path.isdir(processed_data_path):
    for file_name in os.listdir(processed_data_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(processed_data_path, file_name)
            df = pd.read_csv(file_path)
            df_rid = df['rid']
            rid = pd.concat([rid, df_rid])
rid = rid.drop_duplicates(keep='first')
rid = rid.sort_values(by='rid', ascending=True).reset_index(drop=True)


In [ ]:
# Screening Dataset
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr_dataset = Dataset(config_file, scr_file.sheet)
scr = get_df(scr_file.path, scr_file.sheet, config_file).sort_values(by='sc_s1', ascending=True).reset_index(drop=True)

In [ ]:
# Get Screening basic session separated form pei_pre_as and re-screening
# columns_to_drop = ['scspi','nat', 'need_mhpss', 'need_trw', 'need_out_ref', 'need_tmh', 'need_pei', 'note','pei_pre_as', 'sc_re']
columns_to_drop = ['note','pei_pre_as', 'sc_re']
scr_0 = scr.drop(columns=columns_to_drop).dropna(subset=['sc_s1']).sort_values(by='sc_s1')  # Screening basic session. Records = 705
scr_0 = scr_0.drop_duplicates(subset=['rid'], keep='first')  # There are 6 rid duplicated.
re_scr = scr[['rid','sc_re']].dropna()  # Screening re. Records = 41
pei_pre_as = scr[['rid','pei_pre_as']].dropna()  # PEI Assessment. Records = 36

In [ ]:
# pei_pre_as
# re_scr
# scr_0

Note: You need to rearrange the two columns to be in their positions.


In [ ]:
consolidated_dataset_scr = pd.merge(rid, scr_0, on='rid', how='left')
consolidated_dataset_scr = pd.merge(consolidated_dataset_scr, re_scr, on='rid', how='left')
consolidated_dataset_scr = pd.merge(consolidated_dataset_scr, pei_pre_as, on='rid', how='left')

In [ ]:
consolidated_dataset_scr.to_csv('consolidated_dataset.csv', index=False)

In [ ]:
# pd.read_csv('consolidated_dataset.csv')